# NOTE: tải đúng phiên bản các thư viện : 
## TensorFlow: 2.15.0
## Keras: 3.2.1
## KerasNLP: 0.9.3

## Import các thư viện

In [1]:
import matplotlib.pyplot as plt
from keras_nlp.models import DebertaV3Preprocessor
from keras_nlp.models import DebertaV3Classifier
import keras_nlp
import keras
import tensorflow as tf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split

2024-05-27 16:29:31.900605: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 16:29:31.903113: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 16:29:31.939790: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 16:29:32.883670: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Tham khảo: https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter#%F0%9F%8D%BD%EF%B8%8F-%7C-Preprocessing

## Các tham số

In [2]:
# TODO: đổi path file csv
data_path = "train.csv"
# Sử dụng model DeBERTa-V3-Extra-Small-English
model_preset = "deberta_v3_extra_small_en"
max_epochs = 10
batch_size = 5
shufle = 1000
lr = 0.0000003

## Các hàm chính để xử lý model

### 1. Tiền xử lý dữ liệu

#### 1.1 Tiền xử lý label

 Label giá trị trị từ 1-6 (cột score) là dữ liệu categorical.
 Nhưng để làm bài này, label phải là dữ liệu ordinary
 Biến đổi label CATEGORICAL SANG ORDINARY bằng cách :

 Encode label thành một vector nhị phân có độ dài 6, bit i bằng 1 nghĩa là đã thỏa yêu cầu i
 Các giá trị trong vector CHO BIẾT XÁC SUẤT VỊ TRÍ i THỎA YÊU CẦU i.
 Từ đó có thể sử dụng CROSS-ENTROPY để sử dụng hàm lỗi so sánh sự tương đồng giữa 2 phân phối.

 Từ trên ta tạo tính ORDINARY như sau:

 Bài được chấm điểm 1 <= i <= 6 nghĩa là đã thỏa mãn yêu cầu i, VÀ CŨNG THỎA TẤT CẢ CÁC YÊU CẦU TRƯỚC ĐÓ (<= i),
 nên các bit trong vector nhãn sẽ = 1 ở các vị trí <= i, và 0 ở các vị trí còn lại.
 Cách làm này giúp có TÍNH SO SÁNH, vì một khi được điểm j >= i, thì các bit 1 của điểm i cũng = 1 ở trong j


In [3]:
def label_process(y):
    n = len(y)
    z = np.zeros((n, 6), "float32")
    for i in range(n):
        s = y[i]
        z[i, :s] = 1
    return z

#### 1.2 Tiền xử lý đầu vào

Lý do chúng ta cần thực hiện tiền xử lý đầu vào này là vì mô hình  DeBERTa không lấy văn bản thô làm đầu vào. Thay vào đó, chúng làm việc với cách biểu diễn kiểu số cho văn bản. DebertaV3Preprocessor xử lý các bước cần thiết để chuyển đổi văn bản thô thành các biểu diễn số này, bao gồm chuyển sang kiểu token (chia văn bản thành các từ hoặc từ phụ riêng lẻ) và mã hóa các token này dưới dạng số nguyên. Nó cũng đảm bảo rằng tất cả các chuỗi đầu vào đều có cùng độ dài 512, bằng cách đệm các chuỗi ngắn hơn hoặc cắt bớt các chuỗi dài hơn.

In [4]:
preprocessor = DebertaV3Preprocessor.from_preset(
    preset=model_preset,
    sequence_length=512,
)

def input_process(input, label):
    return (preprocessor(input), label)

### 2. Tạo và cấu hình model

Load model đã được cấu hình và train sẵn, sau đó train để fine-tune model này phù hợp cho bài toán. </br>
Như đã nói trên sử đầù ra là một vector độ dài 6, mỗi vị trí cho biết xác suất thỏa yêu cầu i.</br>
Đầu ra của mạng sử dụng sigmoid cho ra giá trị 0->1, chuẩn hóa thành xác suất cho mỗi vị trí i.</br>
Sử dụng hàm Binary Cross-Entropy, vì ta tính xác suất mỗi vị trí (mỗi vị trí đúng (1) đến sai (0)),
sau đó keras.losses.BinaryCrossentropy tổng hợp Loss tại mỗi vị trí làm Loss chung
Sử dụng Adam optimizer kết hợp SGD và momentum

In [5]:
def create_DebertaV3_model(model_preset):
    debertaV3 = DebertaV3Classifier.from_preset(
        model_preset, preprocessor=None, num_classes=6
    )

    inputs = debertaV3.input
    outputs = debertaV3(inputs)

    prob_outputs = keras.layers.Activation("sigmoid")(outputs)
    model = keras.Model(inputs, prob_outputs)

    return model

### 3. Tạo dataset cho dữ liệu train và validation

Load dữ liệu từ file csv và tạo tf.data.Dataset cho dữ liệu train và dữ liệu validation.</br>
Dataset load dữ liệu theo lô (và cache để tiết kiệm bộ nhớ trong) và hoán vị dữ liệu sau mỗi vòng lặp cho THUẬT TOÁN SGD

In [6]:
def create_dataset():
    df = pd.read_csv(data_path)
    X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(
        df["full_text"], df["score"], test_size=0.2, stratify=df["score"]
    )
    X_train = X_train_df.tolist()
    X_val = X_val_df.tolist()
    y_train = label_process(y_train_df.tolist())
    y_val = label_process(y_val_df.tolist())
    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .map(input_process)
        .cache()
        .shuffle(shufle, seed=0)
        .batch(batch_size)
    )
    opt = tf.data.Options()
    opt.experimental_deterministic = False
    train_ds = train_ds.with_options(opt)

    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .map(input_process)
        .cache()
        .batch(batch_size)
    )
    return train_ds, val_ds

## Huấn luyện model

In [8]:
train_ds, val_ds = create_dataset()
model = create_DebertaV3_model(model_preset)

model.compile(
    optimizer=keras.optimizers.Adam(lr),
    loss=keras.losses.BinaryCrossentropy(),
    jit_compile=True,
)
save_model = keras.callbacks.ModelCheckpoint(
    "model.weights.h5",  # HDF5
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="min",
)
# Train model
history = model.fit(
    x = train_ds,
    epochs=max_epochs,
    validation_data=val_ds,
    callbacks=[save_model],
)

Epoch 1/10


I0000 00:00:1716802213.932328   28430 service.cc:145] XLA service 0x7ec4080267c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716802213.932376   28430 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-05-27 16:30:16.524198: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1716802312.194961   28430 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-05-27 16:31:52.249177: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 556911792 exceeds 10% of free system memory.


End of batch:  0
    1/13845 ━━━━━━━━━━━━━━━━━━━━ 531:49:11 138s/step - loss: 0.7125

2024-05-27 16:31:56.074557: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 556911792 exceeds 10% of free system memory.


End of batch:  1
    2/13845 ━━━━━━━━━━━━━━━━━━━━ 12:47:54 3s/step - loss: 0.7203   

...

2024-05-27 16:32:02.466089: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 556911792 exceeds 10% of free system memory.


End of batch:  3
    4/13845 ━━━━━━━━━━━━━━━━━━━━ 12:20:10 3s/step - loss: 0.7218

2024-05-27 16:32:05.603752: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 556911792 exceeds 10% of free system memory.


End of batch:  4
    5/13845 ━━━━━━━━━━━━━━━━━━━━ 12:11:14 3s/step - loss: 0.7220End of batch:  5
    6/13845 ━━━━━━━━━━━━━━━━━━━━ 12:05:31 3s/step - loss: 0.7212End of batch:  6
    7/13845 ━━━━━━━━━━━━━━━━━━━━ 12:03:42 3s/step - loss: 0.7204End of batch:  7
    8/13845 ━━━━━━━━━━━━━━━━━━━━ 11:59:59 3s/step - loss: 0.7195End of batch:  8
    9/13845 ━━━━━━━━━━━━━━━━━━━━ 11:59:23 3s/step - loss: 0.7189End of batch:  9
   10/13845 ━━━━━━━━━━━━━━━━━━━━ 11:59:44 3s/step - loss: 0.7187End of batch:  10
   11/13845 ━━━━━━━━━━━━━━━━━━━━ 11:57:45 3s/step - loss: 0.7189End of batch:  11
   12/13845 ━━━━━━━━━━━━━━━━━━━━ 11:57:06 3s/step - loss: 0.7191End of batch:  12
   13/13845 ━━━━━━━━━━━━━━━━━━━━ 11:54:20 3s/step - loss: 0.7194End of batch:  13
   14/13845 ━━━━━━━━━━━━━━━━━━━━ 11:51:55 3s/step - loss: 0.7198End of batch:  14
   15/13845 ━━━━━━━━━━━━━━━━━━━━ 11:56:00 3s/step - loss: 0.7201

KeyboardInterrupt: 

## Thống kê lỗi và đánh giá

In [ ]:
training_loss = history.history["loss"]
validation_loss = history.history["val_loss"]

epoch_rows = range(1, len(training_loss) + 1)
plt.plot(epoch_rows, training_loss, "r--")
plt.plot(epoch_rows, validation_loss, "b-")
plt.legend(["Training Loss", "Validation Loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.savefig("Loss_plot.png")
plt.show()